# GFT UPV MASTER ML Exercise

#### Purpose of the exercise is to analize the data stored on Elastic , ploting some charts and doing a classification with scikit-learn 

Setup required libraries
-------------------------------------------------------


In [ ]:
pip install elasticsearch
pip install matplotlib
pip install sklearn

Analyze Google stock price evolution
-------------------------------------------------------

1. First thing import Elastic search connector

In [4]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch('elasticsearch:9200')

2. Query all data from quotes index 

In [5]:
res = es.search(index="quotes", body={"query": {"match_all": {}}}, size=10000)
print("Got %d Hits:" % res['hits']['total']['value'])

Got 3106 Hits:


3. Let's print content loaded form elastic quotes index. 

In [ ]:
for hit in res['hits']['hits']:
    print( hit["_source"])

In [ ]:
for hit in res['hits']['hits']:
    print("%(symbol)s %(latestTime)s: %(latestPrice)s" % hit["_source"])

4. Load elastic output as Pandas dataframe

In [ ]:
from pandas.io.json import json_normalize
df = json_normalize(res['hits']['hits'])

df.columns

5. Plot  google stock price evolution

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np

#df2 = df[ (df['_source.symbol']=='GOOG') & (df['_source.latestUpdate']< 1563300000000) &  (df['_source.latestUpdate']> 1563250000000) ]
df2 = df[ (df['_source.symbol']=='GOOG')]
#print( df2['_source.latestUpdate'].apply(lambda x : datetime.fromtimestamp(x/1000.0)))

order = np.argsort(df2['_source.latestUpdate'])

x = np.array(df2['_source.latestUpdate'].apply(lambda x : datetime.fromtimestamp(x/1000.0)))[order]
y = np.array(df2['_source.latestPrice'])[order]
                                      

plt.plot(x,y)

# beautify the x-labels
plt.gcf().autofmt_xdate()

plt.show()


#### Exercise 1: Represent a plot of price change percent over Google and Uber 

Create a clustering over companies sector  and ytd price evolution 
-------------------------------------------------------------------------

1. Load sklearn module and encode companies sector as numbers

In [ ]:
from sklearn import preprocessing

leSector=preprocessing.LabelEncoder()

leSector.fit(df['_source.industry'].astype(str))

leSector.classes_


2. Plot data distribution of sectors and year price evolution

In [ ]:
plt.scatter(df['_source.marketCap'], df['_source.ytdChange'], c='black', s=1)

3. Create a clustering of the data  to  classify companies

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

X = df[['_source.industry','_source.ytdChange']].copy()

X['_source.industry'] = leSector.transform(df['_source.industry'].astype(str))


# Initializing KMeans
kmeans = KMeans(n_clusters=5)
# Fitting with inputs
kmeans = kmeans.fit(X[['_source.industry','_source.ytdChange']])

labels = kmeans.predict(X[['_source.industry','_source.ytdChange']])
# Getting the cluster centers
C = kmeans.cluster_centers_

plt.scatter(X['_source.industry'],X['_source.ytdChange'], c='black', s=1)
plt.scatter(C[:, 0], C[:, 1], marker='*', c='blue', s=100)



#### Exercise 2: Create a classifier over short term stock trending using latestVolume and changePercent

1. Plot the data distribution to identify number of clusters

2. Create a classifier with the number of clusters you  see